# Super-Resolution Convolutional Neural Network

* Reference: https://github.com/sharmaroshan/Image-Super-Resolution/blob/master/Image%20Super%20Resolution%20with%20the%20SRCNN%20(Jupyter%20Notebook).ipynb

In [ ]:
#check package versions
import sys, time
import keras
import cv2
import numpy
import matplotlib
import skimage
import tensorflow as tf

print('Python: {}'.format(sys.version))
print('Keras: {}'.format(keras.__version__))
print('OpenCV: {}'.format(cv2.__version__))
print('NumPy: {}'.format(numpy.__version__))
print('Matplotlib: {}'.format(matplotlib.__version__))
print('Scikit-Image: {}'.format(skimage.__version__))
print('Tensorflow: {}'.format(tf.__version__))

* Reference: https://github.com/sharmaroshan/Image-Super-Resolution/blob/master/Image%20Super%20Resolution%20with%20the%20SRCNN%20(Jupyter%20Notebook).ipynb

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import numpy as np
import math
import os

# python magic function, displays pyplot figures in the notebook
%matplotlib inline

## Image quality metrics

`psnr()`:
* Takes 2 arguments, `target` and `ref`
* First converts the images to float data type to ensure accurate calculations, then it calculates the difference between the reference image and the target image.
    * This difference is flattened into a one-dimensional array using the 'C' style, meaning it is read along rows, like in the C language.
* The Root Mean Squared Error (RMSE) is then calculated by taking the square root of the mean of the squared differences.
* Finally, the PSNR is calculated using the formula `20 * log10(MAX_I / RMSE)`, where `MAX_I` is the maximum possible pixel value of the image. (For an 8-bit RGB image, `MAX_I` is  255)

`mse()`:
* Takes 2 arguments, `target` and `ref`
* Calculates the MSE between the two images, which is the sum of the squared differences between the corresponding pixels in the two images.
    * This sum is then divided by the number of pixels in the target image to get the mean.
* The result is the MSE, which is a measure of the average squared differences between the pixels of the two images.
* Lower MSE values indicate higher similarity between the two images

In [ ]:
# defines function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
        #assume RGB image
        target_data = target.astype(float)
        ref_data = ref.astype(float)
        
        diff = ref_data - target_data
        diff = diff.flatten('C')
        
        rmse = math.sqrt(np.mean(diff ** 2.))
        
        return 20 * math.log10(255. / rmse)

#defines function for mean squared error (MSE)
def mse(target, ref):
        #MSE between two images is the sum of the squared diffference between the two images
        err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
        err /= float(target.shape[0] * target.shape[1])
    
        return err

## Bicubic Interpolation

### Reference: 

* https://www.geeksforgeeks.org/python-opencv-bicubic-interpolation-for-resizing-image/
* https://github.com/rootpine/Bicubic-interpolation/blob/master/bicubic.py

### Interpolation Kernel Function

`interpol_kernel()`:
* Implements an interpolation kernel
* Takes in two parameters: `s` and `a`
    * `s` is the distance from the pixel center
    * `a` is a parameter that controls the shape of the kernel.
* Checks the absolute value of `s` and calculates the kernel value based on the conditions.
* If the absolute value of `s` is between 0 and 1 inclusive, the function calculates the kernel value using the formula: `(a+2)*(abs(s)**3)-(a+3)*(abs(s)**2)+1`
* If the absolute value of `s` is greater than 1 and less than or equal to 2, the function calculates the kernel value using the formula: `a*(abs(s)**3)-(5*a)*(abs(s)**2)+(8*a)*abs(s)-4*a`
* If the absolute value of `s` is greater than 2, the function returns 0 as the kernel value.

In [ ]:
#Interpolation kernel
def interpol_kernel(s, a):
    if (abs(s) >= 0) & (abs(s) <= 1):
        return (a+2)*(abs(s)**3)-(a+3)*(abs(s)**2)+1
    elif (abs(s) > 1) & (abs(s) <= 2):
        return a*(abs(s)**3)-(5*a)*(abs(s)**2)+(8*a)*abs(s)-4*a
    return 0

#### Padding Function

`padding()`:
* Takes four parameters: `img`, `H`, `W`, and `C`
    * `img` is the input image
    * `H` and `W` are the height and width of the image
    * `C` is the number of color channels in the image
* First create a new zero-initialized numpy array `zimg` with dimensions `(H+4, W+4, C)`, which is larger than the input image by 4 pixels in both height and width.
* It then copies the input image into the center of `zimg`, leaving a border of two pixels around the image.
* Next, it pads the first and last two columns and rows of the image by copying the first and last columns and rows of the input image to the corresponding positions in the padded image.
* Finally, it pads the four corners of the image by copying the corner pixels of the input image to the corresponding corner positions in the padded image.
* The result is a new image that is the same as the input image but with a border of two pixels around it. The pixel values in the border are the same as the nearest pixel value in the input image.

In [ ]:
#Padding
def padding(img, H, W, C):
    zimg = np.zeros((H+4, W+4, C))
    zimg[2:H+2, 2:W+2, :C] = img
    
    #pad first/last two col and row
    zimg[2:H+2, 0:2, :C] = img[:, 0:1, :C]
    zimg[H+2:H+4, 2:W+2, :] = img[H-1:H, :, :]
    zimg[2:H+2, W+2:W+4, :] = img[:, W-1:W, :]
    zimg[0:2, 2:W+2, :C] = img[0:1, :, :C]
    
    #pad the missing eight points
    zimg[0:2, 0:2, :C] = img[0, 0, :C]
    zimg[H+2:H+4, 0:2, :C] = img[H-1, 0, :C]
    zimg[H+2:H+4, W+2:W+4, :C] = img[H-1, W-1, :C]
    zimg[0:2, W+2:W+4, :C] = img[0, W-1, :C]
      
    return zimg

### Bicubic Interpolation Function

`bicubic`:
* Takes three parameters: `img`, `ratio`, and `a`.
    * `img` is the input image
    * `ratio` is the scaling factor
    * `a` is a parameter that controls the shape of the interpolation kernel.
* First retrieves the dimensions of the image `(H, W, C)`.
* It then applies padding to the image using the `padding()` function.
* Calculates the dimensions of the output image `(dH, dW)` based on the scaling ratio, and initializes a zero-filled numpy array `dst` of these dimensions.
* Iterates over each color channel and each pixel in the output image. For each pixel, it calculates the corresponding coordinates in the input image `(x, y)`, and the weights for the four nearest horizontal and vertical neighbors using the `interpol_kernel()` function.
* Then it retrieves the pixel values of the 16 nearest neighbors in the input image and calculates the value of the pixel in the output image by taking the dot product of the weights and pixel values.
* If an error occurs during the interpolation process, the function catches the exception, prints an error message, and returns `None`. If the process completes successfully, the function returns the resized image.

In [ ]:
def bicubic(img, ratio, a):
    try:
        #get image size
        H, W, C = img.shape
        #H = height, W = weight, C = number of channels if image is colored
    
        img = padding(img, H, W, C)
    
        #create new image
        dH = math.floor(H*ratio)
        dW = math.floor(W*ratio)
    
        #convert into matrix
        dst = np.zeros((dH, dW, C)) #initialize dst as zero
        #np.zeros generates a matrix consisting of only zeros
    
        h = 1/ratio
    
        print('Starting bicubic interpolation...')
        inc = 0
    
        for c in range(C):
            for j in range(dH):
                for i in range(dW):
                    #get coordinates of nearby values
                    x, y = i * h + 2, j * h + 2
                
                    x_weights = np.array([interpol_kernel(1 + x - math.floor(x), a), interpol_kernel(x - math.floor(x), a),
                                          interpol_kernel(math.floor(x) + 1 - x, a), interpol_kernel(math.floor(x) + 2 - x, a)])
                    y_weights = np.array([interpol_kernel(1 + y - math.floor(y), a), interpol_kernel(y - math.floor(y), a),
                                          interpol_kernel(math.floor(y) + 1 - y, a), interpol_kernel(math.floor(y) + 2 - y, a)])
                
                    # Considering all nearby 16 values
                    x_floor = math.floor(x)
                    y_floor = math.floor(y)
                    pixel_values = img[y_floor-1:y_floor+3, x_floor-1:x_floor+3, c]

                    #calculate the result for this pixel
                    dst[j, i, c] = np.dot(np.dot(x_weights,pixel_values), y_weights)
  
        # If there is an error message, it directly goes to stderr
        sys.stderr.write('\n')
      
        # Flushing the buffer
        sys.stderr.flush()
        return dst
    except Exception as e:
        print(f"Error during bicubic interpolation: {e}")
        return None

## Normalize an image

`normalize()`:
* Takes one parameter: `img` - the input image
* Divides each pixel intensity value in the image by 255.0
    * This is done because the maximum value for a pixel in an 8-bit grayscale image or any color channel in an RGB image is 255. By dividing this maximum value, all pixel values are brought into the range between 0 and 1.
    * This normalized range is often easier for ML algorithms to work with and can help the algorithm converge faster.
* Returns an image with normalized intensities.

In [ ]:
def normalize(img):
    return img / 255.0

## Denormalize an image

`denormalize()`:
* Takes one parameter: `img` - the input image
* Multiplies each pixel intensity value in the image by 255.
    * This reverses the effect of dividing by 255 during normalization.
* Uses the `astype` method to convert the pixel values to the `np.uint8` data type (the standard data type for 8-bit image data).
    * The `astype` method ensures that the pixel values are in the correct format for further image processing.
* Returns a new image with denormalized intensities.

In [ ]:
def denormalize(img):
    return (img * 255).astype(np.uint8)

## Prepare Image for Bicubic Interpolation

`process_image()`:
* Takes four parameters: `img`, `index`, `output_folder`, and optional parameters `ratio` and `a`.
    * `img` is the input image.
    * `index` is used to name the output file.
    * `output_folder` is the directory where the output file will be saved.
    * `ratio` is the scaling factor for the bicubic interpolation.
    * `a` is a parameter that controls the shape of the interpolation kernel.
* It first checks if the input image is `None`, if it is then it raises a `ValueError`.
* Prints the shape of the original image.
* Normalizes the image using the `normalize()` function.
    * scaling the pixel intensity values to the range between 0 and 1.
* Performs bicubic interpolation on the normalized image using the `bicubic()` function.
    * If the interpolation is successful, the function prints a success message.
* It then denormalizes the interpolated image using the `denormalize()` function.
    * Scaling the pixel intensity values back to their original range.
* Checks if the output directory exists. If it doesn't, the function creates it.
* Saves the denormalized image to a file in the output directory, and the file name is generated using the `index` parameter.
* Reads the saved image file and prints its shape.
* Finally, it returns the denormalized image.
* If an error occurs at any point during the process, the function catches the exception, prints an error message, and returns `None`.

In [ ]:
def process_image(img, index, output_folder=None, ratio=2, a=-1/2):
    # Read image
    try:
        if img is None:
            raise ValueError(f"Image could not be loaded.")
        
        # display original image and its shape
        #cv2.imshow('Original Image', img)
        print('Original Image Shape:', img.shape)
        #cv2.waitKey(0) # wait for a key press to close the window
        
        # normalize the image
        normalized_img = normalize(img)

        # Pass the input image in the 
        # bicubic function
        dst = bicubic(normalized_img, ratio, a)
        
        if dst is not None:
            print('Interpolation Completed!')

            # denormalize the interpolated image
            denormalized_dst = denormalize(dst)

            # ensures the output folder exists
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            
            # saving the output image
            output_path = os.path.join(output_folder, f'bicubic_{index}.png')
            cv2.imwrite(output_path, denormalized_dst) 
            print(f"Interpolated image saved as {output_path}")
            
            # display interpolated image and its shape
            if denormalized_dst is not None:
                #cv2.imshow('Bicubic Interpolation Image', denormalized_dst)
                #cv2.waitKey(0) # wait for a key press to close the window
                bicubicImg = cv2.imread(output_path)
                print('Generated Bicubic Image Shape:', bicubicImg.shape)
            
            #cv2.destroyAllWindows() # close all OpenCV windows
            return denormalized_dst
        else:
            print("Bicubic interpolation failed.")
            return None
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

## Load Image Datasets

`load_images()`:
* Takes two parameters: `folder_path` and `max_images`.
    * `folder_path` is the path to the folder containing the image pairs.
    * `max_images` is an optional parameter that specifies the maximum number of image pairs to load. If it is not provided or set to `None`, all image pairs in the folder will be loaded.
* Starts by retrieving the names of all files in the specified folder.
* Initializes two lists, `lr_images` and `hr_images`, to store the low-resolution and high-resolution images respectively.
* If `max_images` is provided, the function reduces the list of file names to twice the number of `max_images`.
    * This is because each image pair consists of two files: one for the low-resolution and one for the high-resolution image.
* It then iterates over the list of file name. For each file name, it constructs the full file path and opens the image file. Converts the image to a numpy array, which is common for image data in ML.
* It appends the image array to the appropriate list, depending on whether the image is a low-resolution or high-resolution image.
    * It determines this based on the index of the file name in the list: even indices correspond to low-resolution images and odd indices correspond to high-resolution images.
* Once all images have been loaded, the function converts the lists of images to numpy arrays and returns them.
    * The first array contains the low-resolution images and the second array contains the high-resolution images.
* If an error occurs while loading the images, the function will raise an exception.

In [ ]:
def load_images(folder_path, max_images=None):
    # Get the file names in the folder
    file_names = sorted(os.listdir(folder_path))

    # Initialize lists to store the LR and HR images
    lr_images = []
    hr_images = []

    # Limit the number of images to load if max_images is set
    if max_images is not None:
        file_names = file_names[:max_images * 2] # times 2 since there are LR and HR pairs

    # Iterate over the file names
    for i, file_name in enumerate(file_names):
        # Exit the loop if the max number of image pairs has been loaded
        if max_images is not None and (i // 2) >= max_images: # // 2 since there are LR and HR pairs
            break

        # Get the file path
        file_path = os.path.join(folder_path, file_name)

        # Open the image and convert it to a numpy array
        image = Image.open(file_path)
        image = np.array(image)

        # Append the image to the appropriate list based on its index (even for LR, odd for HR)
        if i % 2 == 0: # even index, LR image
            lr_images.append(image)
        else:
            hr_images.append(image)

    # Convert the lists to numpy arrays and return them
    return np.array(lr_images), np.array(hr_images)


## Load Training dataset from folder

In [ ]:
max_images_to_load = 5 #ADJUSTABLE NUMBER

#train1_folder = 'G:\My Drive\SR_Verhoef\Synthesis\ImagePairs - Training Data (1 of 14)'
train2_folder = 'G:\My Drive\SR_Verhoef\Synthesis\ImagePairs - Training Data (2 of 14)'

X_train_lr, X_train_hr = load_images(train2_folder, max_images_to_load)

## Resize all LR images

`resize_lr_images()`:
* Takes two parameters: `image_array` and `target_size`.
    * `image_array` is a numpy array containing the batch of images to be resized.
    * `target_size` is a tuple specifying the target height and width for the resized images. If it is not provided, it defaults to `(512, 512)`
* Starts by determining the total number of images in the batch (`num_images`). It then preallocates a numpy array `resized_images` to hold the resized images. The dimensions of this array are `(num_images, *target_size, 3)`, where `*target_size` is the target height and width, and `3` is the number of color channels. The data type of the array is `np.uint8`.
* It then defines the output folder path where the processed images will be saved.
* Next, it enters a loop where it processes each image in the batch. For each image, it calculates a unique index (`unique_index`), which is simply the current index of the loop.
* Calls the `process_image()` function to process the current image by normalizing the image, perform bicubic interpolation to resize the image, denormalize the image, and saves the processed image to a file. The processed file is then saved to the corresponding position in the `resized_images` array.
* Finally, after all images in the batch have been processed the function returns the `resized_images` array containing the resized images.

In [ ]:
def resize_lr_images(image_array, target_size=(512, 512), output_folder_path=None):
    # total number of images
    num_images = len(image_array)
    output_folder = output_folder_path

    # preallocate the array for resized images
    resized_images = np.zeros((num_images, *target_size, 3), dtype=np.uint8)

    if output_folder_path == None:
        output_folder_path = 'G:\My Drive\SR_Verhoef\Analysis\Bicubic Result (training)'

    # process each image in the current batch
    for i, image in enumerate(image_array):
        # calculate a unique index for each image
        unique_index = i

        print("Processing Image....")
        # process the image and save to the array of resized images
        resized_image = process_image(image, unique_index, output_folder)
        resized_images[i] = resized_image

    return resized_images

## Chopping LR images

`chop_lr_images()`:
This functions chops a batch of low-resolution images into smaller segments. It is designed to work with image data in the form of numpy arrays.
* Takes two parameters: `image_array` and `segment_size`.
    * `image_array` is a numpy array containing the batch of images to be chopped.
    * `segment_size` is an integer specifying the height and width of the segments. If it is not provided, it defaults to `255`.
* Starts by initializing an empty list `mini_images` to store the segments.
* Enters a loop where it processes each image in the batch. For each image, it retrieves the height and width of the image. It then calculates the number of segments in the x and y dimensions by integer division of the width and height by the `segment_size`.
* Next, it enters nested loops over the x and y dimensions. For each x and y indices, it extracts a segment from the image. The segment is a square region of the image with size `segment_size`x `segment_size`, starting at the top-left corner `(j*segment_size, i*segment_size)` and ending at the bottom-right corner `((j+1)*segment_size, (i+1)*segment_size)`. It then appends the segment to the `mini_images` list.
* Finally, after all images in the batch have been chopped into segments, the function converts the `mini_images` list to a numpy array and returns it. The returns array contains all the segments from all the images in the batch.

In [ ]:
def chop_lr_images(image_array, segment_size=256):
    #array to store mini images
    mini_images = []

    for img in image_array:
        #Image dimensions
        height, width = img.shape[:2]

        #calculate the number of segments in each dimension
        x_segments = width // segment_size
        y_segments = height // segment_size

        #chop the image into mini images
        for i in range(x_segments):
            for j in range(y_segments):
                mini_img = img[j*segment_size:(j+1)*segment_size, i*segment_size:(i+1)*segment_size]
                mini_images.append(mini_img)

    return np.array(mini_images)

## Chopping HR images

`chop_hr_images()`:
This function has the same process as the `chop_lr_images()` function, only with a different segment size of `512`.

In [ ]:
def chop_hr_images(image_array, segment_size=512):
    #array to store mini images
    mini_images = []

    for img in image_array:
        #Image dimensions
        height, width = img.shape[:2]

        #calculate the number of segments in each dimension
        x_segments = width // segment_size
        y_segments = height // segment_size

        #chop the image into mini images
        for i in range(x_segments):
            for j in range(y_segments):
                mini_img = img[j*segment_size:(j+1)*segment_size, i*segment_size:(i+1)*segment_size]
                mini_images.append(mini_img)

    return np.array(mini_images)

## Build SCRNN Model

`build_srcnn()`:
This function builds the Super-Resolution Convolutional Neural Network (SRCNN) model. 
* Takes two parameters: `input_layer` and `start_neurons`.
    * `input_layer` is the input layer of the model (the input image).
    * `start_neurons` is the number of neurons in the first layer of the model.
* Starts by defining a series of convolutional layers and pooling layers, making up the contracting path of a U-Net Architecture, that downsamples the input image.
    * Each convolutional layer applies a set of filters to the input data, and each pooling layer reduces the spatial dimensions of the data.
    * The number of filters in each convolutional layer is a multiple of `start_neurons`, and the size of the filters decreases from `(7, 7)` to `(3, 3)` as the image is downsampled.
    * The activation function for these layers is the Rectified Linear Unit (ReLU) function, which introduces non-linearity into the model.
* After the downsampling layers, the function defines a 'middle' layer with a larger number of filters.
    * This layer serves as a bottleneck that forces the model to learn a compressed representation of the input data.
* Then it defines a series of upsampling layers that increases the resolution of the image.
    * Each upsampling layer uses the `UpSampling2D` function to double the spatial dimensions of the data, and a `Concatenate` function to concatenate the upsampled data with the corresponding downsampling layer.
    * This is followed by two convolutional layers that refine the upsampled data.
    * The number of filters in these layers decreases from `start_neurons * 16` to `start_neurons * 1`, and the size of the filters increases from `(3, 3)` to `(7, 7)` as the image is upsampled.
* Finally, the function defines an output layer that applies a linear activation function to the data.
    * This layer has 3 filters of size 1, which corresponds to the 3 color channels of the output image.
* The function returns the output layer of the model.

In [ ]:
def build_srcnn(input_layer, start_neurons):

    # 512 -> 256
    conv1a = Conv2D(start_neurons * 1, (7, 7), activation="relu", padding="same")(input_layer)
    conv1b = Conv2D(start_neurons * 1, (7, 7), activation="relu", padding="same")(conv1a)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1b)
    
    # 256 -> 128
    conv2a = Conv2D(start_neurons * 2, (5, 5), activation="relu", padding="same")(pool1)
    conv2b = Conv2D(start_neurons * 2, (5, 5), activation="relu", padding="same")(conv2a)
    pool2 = MaxPooling2D((2, 2))(conv2b)
    
    # 128 -> 64
    conv3a = Conv2D(start_neurons * 4, (5, 5), activation="relu", padding="same")(pool2)
    conv3b = Conv2D(start_neurons * 4, (5, 5), activation="relu", padding="same")(conv3a)
    pool3 = MaxPooling2D((2, 2))(conv3b)
    
    # 64 -> 32
    conv4a = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4b = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4a)
    pool4 = MaxPooling2D((2, 2))(conv4b)

    # 32 -> 16
    conv5a = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    conv5b = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(conv5a)
    pool5 = MaxPooling2D((2, 2))(conv5b)
        
    #middle
    convm1 = Conv2D(start_neurons * 32, (3, 3), activation="relu", padding="same")(pool5)
    convm2 = Conv2D(start_neurons * 32, (3, 3), activation="relu", padding="same")(convm1)
        
    #Upsampling
    # 16 -> 32
    up1 = UpSampling2D(size=(2, 2))(convm2)
    concat1 = Concatenate(axis=3)([up1, conv5b])
    conv6a = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(concat1)
    conv6b = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(conv6a)
    
    # 32 -> 64
    up2 = UpSampling2D(size=(2, 2))(conv6b)
    concat2 = Concatenate(axis=3)([up2, conv4b])
    conv7a = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(concat2)
    conv7b = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv7a)
    
    # 64 -> 128
    up3 = UpSampling2D(size=(2, 2))(conv7b)
    concat3 = Concatenate(axis=3)([up3, conv3b])
    conv8a = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(concat3)
    conv8b = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv8a)
   
    # 128 -> 256
    up4 = UpSampling2D(size=(2, 2))(conv8b)
    concat4 = Concatenate(axis=3)([up4, conv2b])
    conv9a = Conv2D(start_neurons * 2, (5, 5), activation="relu", padding="same")(concat4)
    conv9b = Conv2D(start_neurons * 2, (5, 5), activation="relu", padding="same")(conv9a)

    # 256 -> 512
    up5 = UpSampling2D(size=(2, 2))(conv9b)
    concat5 = Concatenate(axis=3)([up5, conv1b])
    conv10a = Conv2D(start_neurons * 1, (7, 7), activation="relu", padding="same")(concat5)
    conv10b = Conv2D(start_neurons * 1, (7, 7), activation="relu", padding="same")(conv10a)

    #Output Layer
    output_layer = Conv2D(3, 1, activation='linear')(conv10b)

    return output_layer

## Create an Instance of this Model and Compile it

This code is used to create an compile a SRCNN model.
* Starts by defining the shape of the input data, which is set to `(512, 512, 3)`.
* Next, an input layer is created using the `Input` function from Keras. The shape of the input layer is set to the `input_shape` defined earlier.
* It then sets the number of neurons in the first layer of the SRCNN model to 32. This is an adjustable value that can be tuned based on the specific requirments.
* The SRCNN model is then instantiated by calling the `build_srcnn` function with the input layer and the number of starting neurons as arguments.
* A Keras Model is then created using the `Model` function. The inputs to the model are set to the input layer, and the outputs are set to the output layer of the SRCNN model.
* The model is then compiled using the `Adam` optimizer with a learning rate of 0.001, and the mean squared error loss function. The optimizer and loss function are set using the `compile` method of the Keras model.
* Finally, the `summary` method is called on the model to print a summary of the model's architecture. This includes the number of layers in the model, the shape of the output from each layer, and the number of parameters in each layer.

#### Instance of Model and Compilation

In [ ]:
#Adjust size accordingly
input_shape = (512, 512, 3)

#create input layer
input_layer = Input(shape=input_shape)

#instantiate model
start_neurons = 32 #adjustable value
sr_model = build_srcnn(input_layer, start_neurons)

#create the model
model = Model(inputs=input_layer, outputs=sr_model)

#compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

#summary of the model
model.summary()

#### Code Explained (TTS)

* 'train_test_split' is used to split the datasets into training and testing (validation) sets.
* 'X_train_lr' and 'X_train_hr' are the previously loaded and split LR and HR image arrays

## Chop LR and HR Images, then resize LR images with bicubic interpolation

In [ ]:
output_folder_path = 'G:\My Drive\SR_Verhoef\Analysis\Bicubic Result (training)'

lr_segments = chop_lr_images(X_train_lr)
hr_segments = chop_hr_images(X_train_hr)
#print(lr_segments)
# Resize the lr_segments with bicubic interpolation method
lr_segments = resize_lr_images(lr_segments, output_folder_path=output_folder_path)

print("Size of lr_segments: ", lr_segments.shape)
# Now, lr_segments and hr_segments are 2D arrays ready for training


## Train, Test, Split chopped images

In [ ]:
# lr_images.shape = (num_samples, height, width, channels)
# hr_images.shape = (num_samples, height, width, channels)

#split training data into training and validation sets
X_train_lr, X_val_lr, X_train_hr, X_val_hr = train_test_split(
    lr_segments,
    hr_segments,
    test_size=0.2, # 20% of the data will be used as the validation set
    random_state=42, # ensures reproducibility of the split
    shuffle=False # ensures the data isnt shuffled
)

## Training the Model

* Defines three callbacks used during the training process, applied at different stages.
    * `ModelCheckpoint` saves the model after every epoch.
    * `save_best_only` is set to `True`, meaning that the model is only saved if its performance on the validation set has improved since the last epoch.
    * `ReduceLROnPlateau` reduces the learning rate when a metric has stopped improving.
        * `factor` = 0.1, means the learning rate will reduce by a factor of 10.
        * `patience` = 5, means that the learning rate will be reduced if the metric has not improved for 5 epochs.
        * `min_lr` = 0.00001, is the lower bound on the learning rate.
    * `EarlyStopping` stops training when a monitored metric has stopped improving.
        * `patience` = 10, meaning the training will be stopped if the metric has not improved after 10 epochs.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

#Callbacks
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(patience=10, verbose=1)

#fit the model
histroy = model.fit(
    x=X_train_lr, # input low-resolution images
    y=X_train_hr, # corresponding high-resolution images
    validation_data=(X_val_lr, X_val_hr),
    epochs=25, # number of epochs to train
    batch_size=28, # number of samples per gradient update
    callbacks=[checkpoint, reduce_lr, early_stopping],
    verbose=1
)

## Test 1

In [ ]:
# Load test data
test1_folder = 'G:\My Drive\SR_Verhoef\Synthesis\ImagePairs - Test Data (1 of 5)'
bi_output_folder = 'G:\My Drive\SR_Verhoef\Analysis\Bicubic Interpolated Image Outputs (Testing)'
max_images_to_load = 1

# Split test data into lists
Y_test_lr, Y_test_hr = load_images(test1_folder, max_images_to_load)

# Chop images
Y_test_lr = chop_lr_images(Y_test_lr)
Y_test_hr = chop_hr_images(Y_test_hr)

# Resize lr images
Y_test_lr = resize_lr_images(Y_test_lr, output_folder_path=bi_output_folder)

recombined_bi_image = np.block(Y_test_lr)

# Saving the output image
output_path = os.path.join(bi_output_folder, 'bicubic_final.png')
cv2.imwrite(output_path, recombined_bi_image)
print('Bicubic image saved!')

In [ ]:
#evaluate quality of resized lr images with psnr and mse
bi_mse_values = []
bi_psnr_values = []

for i in range(len(Y_test_lr)):
    bi_mse_value = mse(Y_test_hr[i], Y_test_lr[i])
    bi_psnr_value = psnr(Y_test_hr[i], Y_test_lr[i])
    bi_mse_values.append(bi_mse_value)
    bi_psnr_values.append(bi_psnr_value)

#Calculate average MSE and PSRN
avg_mse_bi = np.mean(bi_mse_values)
avg_psnr_bi = np.mean(bi_psnr_values)

#Calculate standard deviation of MSE and PSNR
bi_std_mse = np.std(bi_mse_values)
bi_std_psnr = np.std(bi_psnr_values)

print("Bicubic Average MSE: ", avg_mse_bi)
print("Bicubic Average PSNR: ", avg_psnr_bi)
print("Bicubic Standard Deviation of MSE: ", bi_std_mse)
print("Bicubic Standard Deviation of PSNR: ", bi_std_psnr)

In [ ]:
Y_pred = model.predict(Y_test_lr)
# Calculate MSE and PSNR
mse_values = []
psnr_values = []

for i in range(len(Y_test_lr)):
    mse_value = mse(Y_test_hr[i], Y_pred[i])
    psnr_value = psnr(Y_test_hr[i], Y_pred[i])
    mse_values.append(mse_value)
    psnr_values.append(psnr_value)

# Calculate average MSE and PSNR
avg_mse = np.mean(mse_values)
avg_psnr = np.mean(psnr_values)

# Calculate standard deviation of MSE and PSNR
std_mse = np.std(mse_values)
std_psnr = np.std(psnr_values)

print("SR Average MSE: ", avg_mse)
print("SR Average PSNR: ", avg_psnr)
print("SR Standard Deviation of MSE: ", std_mse)
print("SR Standard Deviation of PSNR: ", std_psnr)


In [ ]:
final_output_folder = 'G:\My Drive\SR_Verhoef\Analysis\Final SRCNN ouput'

# Create the output folder if it doesn't exist
if not os.path.exists(final_output_folder):
    os.makedirs(final_output_folder)

# Save the super-resolution images
for i in range(len(Y_pred)):
    # Convert the image data to 8-bit per channel
    img = np.clip(Y_pred[i], 0, 255).astype('uint8')

    # Save the image with a formatted file name
    file_name = f'output_image_{i}.png'
    file_path = os.path.join(final_output_folder, file_name)
    cv2.imwrite(file_path, img)


In [ ]:
#recombine the final sr output
recombined_sr_image = np.block(Y_pred)
# saving the output image
output_path = os.path.join(final_output_folder, f'bicubic_final.png')
cv2.imwrite(output_path, recombined_sr_image)
print('Final SR image saved!')